<a href="https://colab.research.google.com/github/DanB1421/DATA602/blob/main/Week_11_Problem_Set%20%20%20%20%20%20%20%20%20%20%20%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rock Paper Scissors is a classic hand game where players compete through using hand gestures (see [Wikipedia](https://en.wikipedia.org/wiki/Rock\_paper\_scissors)).  The Tensorflow-datasets package provides a
library of hand gestures corresponding to allowed gestures in the game.  The original source is a synthetic data source, via Laurence Maroney ([source](https://laurencemoroney.com/datasets.html\#rock-paper-scissors-dataset)).

Write a convolutional neural network (CNN) that classifies these images, i.e., identifies the gesture (rock, paper, scissors) from each source image.  For convenience, this template is provided for loading the dataset and for data augmentation.  Feel free to make any changes to this template code.  Try to achieve a validation accuracy rate of 90\% or above.

You may make use of pretrained CNN architectures in Keras or develop your own.  If you develop your own, you are encouraged to review the network architectures discussed in class (e.g., LeNet and AlexNet).  If you use a pretrained architecture, refer to the [Transfer Learning and fine-tuning](https://keras.io/guides/transfer_learning/) developer guide on the Keras website.  

Because the training dataset is relatively small, you may need to employ techniques such as regularization and data augmentation to achieve a high accuracy rate.  Code for data augmentation is included in the template.

Using a graphics card is highly recommended when fitting the model.  You can set Colab to use the graphics card in Runtime $\rightarrow$ Manage Settings.

# Preamble

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from functools import partial
import numpy as np

#Data loading and preprocessing

We first load the test dataset from tensorflow datasets (TFDS).  The original set has only training and validation sets, so we shuffle the dataset here to produce training, validation, and validation sets.

* [Documentation for `tfds.load`](https://www.tensorflow.org/datasets/api_docs/python/tfds/load)
* [Rock Paper Scissors dataset](https://www.tensorflow.org/datasets/catalog/rock_paper_scissors)
* [TFDS images in this dataset](https://knowyourdata-tfds.withgoogle.com/#tab=STATS&dataset=rock_paper_scissors)

In [ ]:
(ds_orig, ds_val_orig, ds_test_orig), dsinfo = tfds.load(
    'rock_paper_scissors',
    split=("test+train[60%:]", "train[60%:80%]", "train[80%:]"),
    with_info=True,
    as_supervised=True,
    shuffle_files=False)

labelnames = dsinfo.features['label'].names

Below are constants, configure as needed.

In [ ]:
batch_size = 8   # size of mini-batch
image_size = 128  # size of image to feed into network
n_repeat = 5 # number of times to repeat the input dataset with distorted images

We create a convenience function, `show_sample_images` to show 10 random images from the set.  Example usage: `show_sample_images(ds_orig)`.

In [ ]:
def show_sample_images(ds):
  fig, axs = plt.subplots(4, 4, figsize=(10,10))
  for ax, k in zip(np.array(axs).ravel(), ds.unbatch().shuffle(128).take(16)):
      ax.imshow(k[0])
      ax.set_title(labelnames[k[1]])
  fig.tight_layout()

We will need to perform some preprocessing.  The below preprocessing function is a starting point.

* `distort` is a network that performs image augmentation.  For input images, it will randomly provide distortions such as distorting contrast, zooming in or out, rotation, and translation.  Passing an image into this network will apply these distortions.  Refer to documentation for these functions [here](https://www.tensorflow.org/api_docs/python/tf/image).

* `preproc` is a preprocessing function that accepts an image and label, and can apply updates to both (similar to a FunctionTransformer in scikit).  We only apply distortions if augment is true (as we will see, we want to apply to the training dataset).  

   Preproc also converts the image to grayscale, which provides some performance improvement since color is not expected important for processing.  If you remove grayscale conversion, remember to change the depth of the input layer from 1 to 3.

  Finally, preproc rescales the image from 0-255 to 0-1 ([`convert_image_dtype`](https://www.tensorflow.org/api_docs/python/tf/image/convert_image_dtype)) and resizes the image.

  * We also create two wrapper functions, `preproc_no_distort` and `preproc_distort`, for later compatibility with the tensor flow's data object.

In [ ]:
distort = keras.Sequential(
    [
    keras.layers.RandomContrast(0.2),
    keras.layers.RandomZoom((-.05, .05), fill_mode='nearest'),
    keras.layers.RandomRotation((-.04, .04), fill_mode='nearest'),
    keras.layers.RandomTranslation(0, (-.2, .2), fill_mode='nearest')
    ]
)
def preproc(image, label, augment):
  if augment:
    image = distort(image)/255.0
  #image = tf.image.rgb_to_grayscale(image)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, (image_size, image_size))
  return image, label

def preproc_no_distort(image, label):
  return preproc(image, label, False)

def preproc_distort(image, label):
  return preproc(image, label, True)

We can now prepare the datasets.  For the three datasets defined above (training, validation, and test), we preprocess each element using the `map` function.  (See [guidelines for TensorFlow input pipelines](https://www.tensorflow.org/guide/data) and [`tf.data.Dataset` documentation](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).  From the texts, see also chapter 13 in Raschka and chapter 11 in Geron.  We also break the training set into minibatches.


In [ ]:
ds = ds_orig.map(preproc_distort).batch(batch_size)
ds_val = ds_val_orig.map(preproc_no_distort).batch(1)
ds_test = ds_test_orig.map(preproc_no_distort).batch(1)

# Build Model

In [ ]:
# Write and compile the model here

pretrained_model = keras.applications.resnet50.ResNet50(include_top=False,
                            input_shape=(image_size, image_size, 3),
                            pooling="max")
pretrained_model.trainable = False

model = Sequential([
    pretrained_model,
    Dense(1024, activation="elu"),
    Dense(3, activation="softmax")
])

model.compile(optimizer=keras.optimizers.Nadam(),
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])
print(model.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_16 (Dense)            (None, 1024)              2098176   
                                                                 
 dense_17 (Dense)            (None, 3)                 3075      
                                                                 
Total params: 25,688,963
Trainable params: 2,101,251
Non-trainable params: 23,587,712
_________________________________________________________________
None


# Fit

We have two callbacks:

* `ModelCheckpoint`, which saves the model after each mini-batch.  We set the `save_best_only` parameter so that if the model performance deteriorates, we only keep the weights from the best-performing batch.

* `EarlyStopping`, which stops the model after no improvement for four iterations.

Note that we repeat the dataset here by calling `ds.repeat(n_repeat)`.  Because we are using image augmentation, this allows the network to be trained on multiple variants of each distorted image.

**Remember to change the runtime type to GPU before running (Change Runtime Type under the Tools menu.)**  You will need to re-run the notebook after changing the runtime type.

In [ ]:
earlystopping = keras.callbacks.EarlyStopping(min_delta=0.005, patience=4)
checkpoint = keras.callbacks.ModelCheckpoint("mdl-rps-eval-gap.h5", save_best_only=True)
dsr = ds.repeat(n_repeat) if n_repeat > 0 else ds
history = model.fit(dsr, validation_data=ds_val, epochs=20,
                    callbacks=[earlystopping, checkpoint])


Epoch 1/20
865/865 [==============================] - 183s 205ms/step - loss: 0.3247 - accuracy: 0.8732 - val_loss: 0.0640 - val_accuracy: 0.9881
Epoch 2/20
865/865 [==============================] - 178s 206ms/step - loss: 0.1753 - accuracy: 0.9336 - val_loss: 0.0185 - val_accuracy: 0.9960
Epoch 3/20
865/865 [==============================] - 175s 203ms/step - loss: 0.1553 - accuracy: 0.9436 - val_loss: 0.0876 - val_accuracy: 0.9722
Epoch 4/20
865/865 [==============================] - 176s 203ms/step - loss: 0.1399 - accuracy: 0.9516 - val_loss: 0.0166 - val_accuracy: 0.9960
Epoch 5/20
865/865 [==============================] - 175s 202ms/step - loss: 0.1280 - accuracy: 0.9535 - val_loss: 0.0479 - val_accuracy: 0.9802
Epoch 6/20
865/865 [==============================] - 174s 201ms/step - loss: 0.1130 - accuracy: 0.9577 - val_loss: 0.1250 - val_accuracy: 0.9603


#  Evaluate

We evaluate the best-performing model (saved in the model checkpoint callback) against the test dataset.  Recall that we are looking for an accuracy of 0.9 or above.

In [ ]:
model = keras.models.load_model("mdl-rps-eval-gap.h5")
_, accuracy = model.evaluate(ds_test)
print(accuracy)

504/504 [==============================] - 7s 11ms/step - loss: 0.0237 - accuracy: 0.9901
0.9900793433189392
